In [1]:
import os
from datetime import datetime, timedelta

import gnome
from gnome import scripting
import gnome.scripting as gs
import netCDF4

import numpy as np
from numpy import inf

from gnome import scripting
from gnome import utilities
from gnome.utilities import profiledeco as pd
from gnome.utilities.remote_data import get_datafile
from gnome.basic_types import datetime_value_2d, numerical_methods

from gnome.utilities.remote_data import get_datafile

from gnome.model import Model

from gnome.maps import MapFromBNA
from gnome.spills import surface_point_line_spill
from gnome.spills import PointLineRelease,Spill
from gnome.movers import RandomMover, GridWindMover, c_GridCurrentMover, constant_wind_mover, WindMover

from gnome.outputters import Renderer
from gnome.outputters.animated_gif import Animation
from gnome.outputters import NetCDFOutput
from gnome.outputters import ShapeOutput
from gnome.environment import constant_wind, Water, Waves, Wind
from gnome.weatherers import (Emulsification,
                              Evaporation,
                              NaturalDispersion,
                              WeatheringData)

# Defindo diretorio base
base_dir = os.path.dirname("__file__")


def make_model(images_dir=os.path.join(base_dir, 'images')):
    print('Inicializando o modelo')

    start_time = datetime(2017, 7, 1, 12, 0)

### Configurando o modelo
    model = Model(start_time=start_time,
                  duration=timedelta(days = 30),
                  time_step=3600, uncertain=True)

### Adicionando mapa 
    mapfile = get_datafile(os.path.join(base_dir, 'TCCcoast.bna'))

    print('Adicionando mapa')
    model.map = MapFromBNA(mapfile, refloat_halflife=1)  

    # 'forecast' LEs are in black, and 'uncertain' are in red

    renderer = Renderer(mapfile, images_dir, image_size=(600, 600),
                        output_timestep=timedelta(hours=12),
                        timestamp_attrib={'size': 'medium', 'color':'uncert_LE'})
    renderer.set_timestamp_attrib(format='%a %c')
    renderer.graticule.set_DMS(True)


    print('Adding spill')
    end_time = start_time + gs.hours(720) 
    oil_file = os.path.join(base_dir, 'loreto_AD01688.json')
    substance = gs.GnomeOil(filename=oil_file)
    release = PointLineRelease(release_time=start_time,end_release_time=end_time, start_position=(-46.52,-26.46,0))
    spill = Spill(release)
    spill = surface_point_line_spill(num_elements=1000,
                                     amount = 1092.56,
                                     start_position=(-46.52,-26.46,0),
                                     release_time=start_time,
                                     end_release_time=end_time,
                                     substance=substance,
                                     units='m^3',
                                     windage_range = (0.01,0.04))
    #wind_persist: set default

    model.spills += spill

    print('Adding RandomMover:')
    model.movers += RandomMover(diffusion_coef=10000)
    
    water = Water (temperature=294.8935, salinity=35.0,wave_height=1.91505, units = {'temperature': 'K', 'salinity': 'psu', 'wave_height': 'm'})
    wind = constant_wind(7.89857, 228.621, units = 'm/s') 
    waves = Waves (wind, water)

    print('Adding grid wind:')

    w_mover = gs.PyWindMover('tcc_era5.nc')

    model.movers += w_mover

 
    print('Adding current mover:')
    curr_file = gs.get_datafile(os.path.join(base_dir, 'hycom_season_avg.nc'))
    c_mover = gs.PyCurrentMover(curr_file)
    model.movers += c_mover

    model.environment += water
    model.environment += wind
    model.environment += waves

    model.weatherers += Evaporation(water, wind)
    model.weatherers += Emulsification(waves)
    model.weatherers += NaturalDispersion(waves, water)


    print('Adding outputters')
    model.outputters += renderer

## netcdf
    netcdf_file = os.path.join(base_dir, 'saida.nc')
    gs.remove_netcdf(netcdf_file)
    model.outputters += gs.NetCDFOutput(netcdf_file, which_data='all')

## KMZ
    #model.outputters += gs.KMZOutput(os.path.join(base_dir, 'saida.kmz'))

## ShapeFile
    model.outputters += gs.ShapeOutput(os.path.join(base_dir, 'shape'))
    
## Wheathering
    model.outputters += gs.WeatheringOutput(os.path.join(base_dir, 'output'))
    
    return model


if __name__ == "__main__":
    startTime = datetime.now()
    scripting.make_images_dir()
    model = make_model()
    print("doing full run")
    print("Note: Images folder and animation do not create same output, for demonstration")
    anim = model.outputters[0]
    model.map.save_as_image('raster.bmp')
    for step in model:
        if step['step_num'] == 12:
            anim.set_timestamp_attrib(on=False)
        if step['step_num'] == 24:
            anim.set_timestamp_attrib(on=True)
        print("step: %.4i -- memuse: %fMB" % (step['step_num'],
                                              utilities.get_mem_use()))
    print(datetime.now() - startTime)

('images_dir is:', 'C:\\Users\\franh\\Downloads\\TCC\\Git\\PyGnome\\py_gnome\\Experimentos tcc\\inverno\\images')
Inicializando o modelo
Adicionando mapa
Adding spill
Adding RandomMover:
Adding grid wind:
Adding current mover:
Adding outputters
doing full run
Note: Images folder and animation do not create same output, for demonstration
step: 0000 -- memuse: 160.851562MB


c:\users\franh\downloads\tcc\git\pygnome\py_gnome\gnome\environment\gridded_objects_base.py:618: UserWarning: Variable0 units is not supported: m s**-1Using them unconverted as m/s
  warnings.warn("{0} units is not supported: {1}"


step: 0001 -- memuse: 162.796875MB
step: 0002 -- memuse: 162.976562MB
step: 0003 -- memuse: 169.992188MB
step: 0004 -- memuse: 170.191406MB
step: 0005 -- memuse: 170.191406MB
step: 0006 -- memuse: 170.191406MB
step: 0007 -- memuse: 170.257812MB
step: 0008 -- memuse: 170.390625MB
step: 0009 -- memuse: 167.929688MB
step: 0010 -- memuse: 170.769531MB
step: 0011 -- memuse: 170.835938MB
step: 0012 -- memuse: 170.968750MB
step: 0013 -- memuse: 171.171875MB
step: 0014 -- memuse: 171.167969MB
step: 0015 -- memuse: 171.167969MB
step: 0016 -- memuse: 171.234375MB
step: 0017 -- memuse: 171.238281MB
step: 0018 -- memuse: 171.238281MB
step: 0019 -- memuse: 171.238281MB
step: 0020 -- memuse: 171.238281MB
step: 0021 -- memuse: 171.679688MB
step: 0022 -- memuse: 171.878906MB
step: 0023 -- memuse: 172.281250MB
step: 0024 -- memuse: 172.347656MB
step: 0025 -- memuse: 172.593750MB
step: 0026 -- memuse: 172.726562MB
step: 0027 -- memuse: 172.824219MB
step: 0028 -- memuse: 172.988281MB
step: 0029 -- memuse

step: 0236 -- memuse: 178.488281MB
step: 0237 -- memuse: 177.859375MB
step: 0238 -- memuse: 177.871094MB
step: 0239 -- memuse: 178.468750MB
step: 0240 -- memuse: 178.492188MB
step: 0241 -- memuse: 178.417969MB
step: 0242 -- memuse: 178.480469MB
step: 0243 -- memuse: 178.484375MB
step: 0244 -- memuse: 178.484375MB
step: 0245 -- memuse: 177.531250MB
step: 0246 -- memuse: 177.570312MB
step: 0247 -- memuse: 177.589844MB
step: 0248 -- memuse: 177.589844MB
step: 0249 -- memuse: 177.621094MB
step: 0250 -- memuse: 177.351562MB
step: 0251 -- memuse: 177.406250MB
step: 0252 -- memuse: 177.656250MB
step: 0253 -- memuse: 177.656250MB
step: 0254 -- memuse: 177.656250MB
step: 0255 -- memuse: 177.964844MB
step: 0256 -- memuse: 177.964844MB
step: 0257 -- memuse: 177.964844MB
step: 0258 -- memuse: 177.593750MB
step: 0259 -- memuse: 177.601562MB
step: 0260 -- memuse: 177.660156MB
step: 0261 -- memuse: 178.015625MB
step: 0262 -- memuse: 177.789062MB
step: 0263 -- memuse: 178.183594MB
step: 0264 -- memuse

step: 0471 -- memuse: 181.414062MB
step: 0472 -- memuse: 181.437500MB
step: 0473 -- memuse: 181.437500MB
step: 0474 -- memuse: 178.265625MB
step: 0475 -- memuse: 181.464844MB
step: 0476 -- memuse: 181.503906MB
step: 0477 -- memuse: 181.437500MB
step: 0478 -- memuse: 177.605469MB
step: 0479 -- memuse: 178.027344MB
step: 0480 -- memuse: 181.703125MB
step: 0481 -- memuse: 181.621094MB
step: 0482 -- memuse: 181.656250MB
step: 0483 -- memuse: 181.679688MB
step: 0484 -- memuse: 177.546875MB
step: 0485 -- memuse: 178.082031MB
step: 0486 -- memuse: 177.828125MB
step: 0487 -- memuse: 181.667969MB
step: 0488 -- memuse: 181.703125MB
step: 0489 -- memuse: 178.289062MB
step: 0490 -- memuse: 181.667969MB
step: 0491 -- memuse: 177.375000MB
step: 0492 -- memuse: 181.699219MB
step: 0493 -- memuse: 178.625000MB
step: 0494 -- memuse: 178.097656MB
step: 0495 -- memuse: 177.796875MB
step: 0496 -- memuse: 177.691406MB
step: 0497 -- memuse: 181.621094MB
step: 0498 -- memuse: 177.601562MB
step: 0499 -- memuse

step: 0706 -- memuse: 184.132812MB
step: 0707 -- memuse: 180.207031MB
step: 0708 -- memuse: 179.210938MB
step: 0709 -- memuse: 184.160156MB
step: 0710 -- memuse: 180.472656MB
step: 0711 -- memuse: 180.601562MB
step: 0712 -- memuse: 184.281250MB
step: 0713 -- memuse: 179.167969MB
step: 0714 -- memuse: 181.773438MB
step: 0715 -- memuse: 184.281250MB
step: 0716 -- memuse: 184.406250MB
step: 0717 -- memuse: 180.066406MB
step: 0718 -- memuse: 181.039062MB
step: 0719 -- memuse: 183.960938MB
step: 0720 -- memuse: 180.574219MB
0:11:34.884401
